# main

In [1]:
!python main.py

100%|████████████████████████████████████| 60578/60578 [01:00<00:00, 996.72it/s]


# check output

In [3]:
!python check_lmdb.py

60548
(482, 287)
{'axes': {'x-axis': {'tick-type': 'markers',
                     'ticks': [{'id': 10, 'tick_pt': {'x': 104, 'y': 179}},
                               {'id': 11, 'tick_pt': {'x': 130, 'y': 179}},
                               {'id': 12, 'tick_pt': {'x': 156, 'y': 179}},
                               {'id': 13, 'tick_pt': {'x': 181, 'y': 179}},
                               {'id': 14, 'tick_pt': {'x': 206, 'y': 179}},
                               {'id': 15, 'tick_pt': {'x': 232, 'y': 179}},
                               {'id': 16, 'tick_pt': {'x': 258, 'y': 179}},
                               {'id': 17, 'tick_pt': {'x': 284, 'y': 179}},
                               {'id': 18, 'tick_pt': {'x': 309, 'y': 179}},
                               {'id': 19, 'tick_pt': {'x': 334, 'y': 179}},
                               {'id': 20, 'tick_pt': {'x': 360, 'y': 179}},
                               {'id': 21, 'tick_pt': {'x': 386, 'y': 179}},
                          

# appendix

In [81]:
import re
from pprint import pprint

def anns2point_coors(json_dict):
    tick_id2text = {text_dict['id']: text_dict['text'] for text_dict in json_dict['text']}
    
    axis_dict = json_dict['axes']
    
    tick2coor = {}
    
    # horizontalの場合は特殊
    if json_dict['chart-type'] == 'horizontal_bar':
        x_ticks_temp = axis_dict['x-axis']['ticks']
        axis_dict['x-axis']['ticks'] = axis_dict['y-axis']['ticks']
        axis_dict['y-axis']['ticks'] = x_ticks_temp
    # dotはxがcategoricalっぽいのにnumericalとか言ってる
    if json_dict['chart-type'] == 'dot':
        json_dict['axes']['x-axis']['values-type'] = 'categorical'

    def _fix_text(text):
        text = str(text)
        print(text)
        text = re.sub('.$', '', text)
        text = text.replace(',', '').replace('%', '').replace('..', '.').replace('$', '')
        if len(text) == 0:
            return 0.0
        else:
            return float(text)
    
    for ax_key, ax_dict in axis_dict.items():
        ax_type = ax_key.split('-')[0] # x or y
        if ax_dict['values-type'] == 'numerical':
            coors, values = [], []
            for tick_dict in ax_dict['ticks']:
                tick_id = tick_dict['id']
                text = tick_id2text[tick_id]
                coor = tick_dict['tick_pt'][ax_type]
                
                values.append(_fix_text(text))
                coors.append(coor)
            if len(coors) == 0 or len(values) == 0:
                a = b = 0
            else:
                a = (coors[-1] - coors[0]) / (values[-1] - values[0])
                b = coors[-1] - a * values[-1]
            tick2coor[ax_type] = {'values-type': 'numerical', 'convert': (a, b)}
            
        elif ax_dict['values-type'] == 'categorical':
            text2coor = {}
            for tick_dict in ax_dict['ticks']:
                tick_id = tick_dict['id']
                text = tick_id2text[tick_id]
                coor = tick_dict['tick_pt'][ax_type]
                text2coor[text] = coor
            tick2coor[ax_type] = {'values-type': 'categorical', 'convert': text2coor}
    ann_coors = []
    for ann in json_dict['data-series']:
        ann_coor = {}
        data = {'x': ann['x'], 'y':ann['y']}
        for ax_type, ax_tick2coor in tick2coor.items():
            try:
                if ax_tick2coor['values-type'] == 'numerical':
                    a, b = ax_tick2coor['convert']
                    ann_coor[ax_type] = a * data[ax_type] + b
                else:
                    text2coor = ax_tick2coor['convert']
                    ann_coor[ax_type] = text2coor[data[ax_type]]
            except:
                pass
        ann_coors.append(ann_coor)
    return ann_coors

def check_ann(id_):
    from pathlib import Path
    import json
    ROOT_DIR = Path.cwd().parents[2]
    RAW_DATA_DIR = ROOT_DIR / 'original_data'
    TRAIN_LABEL_DIR = RAW_DATA_DIR / 'train' / 'annotations'
    json_path = TRAIN_LABEL_DIR / (id_ + '.json')
    with open(json_path, 'r') as f:
        json_dict = json.load(f)
    print(anns2point_coors(json_dict))

def pprint_json(id_):
    from pathlib import Path
    import json
    ROOT_DIR = Path.cwd().parents[2]
    RAW_DATA_DIR = ROOT_DIR / 'original_data'
    TRAIN_LABEL_DIR = RAW_DATA_DIR / 'train' / 'annotations'
    json_path = TRAIN_LABEL_DIR / (id_ + '.json')
    with open(json_path, 'r') as f:
        json_dict = json.load(f)
    pprint(json_dict)

In [85]:
check_ann('2882a5757ecf')

1.000.000


ValueError: could not convert string to float: '1.000.00'

In [84]:
pprint_json('2882a5757ecf')

{'axes': {'x-axis': {'tick-type': 'markers',
                     'ticks': [{'id': 14, 'tick_pt': {'x': 115, 'y': 238}},
                               {'id': 15, 'tick_pt': {'x': 139, 'y': 238}},
                               {'id': 16, 'tick_pt': {'x': 162, 'y': 238}},
                               {'id': 17, 'tick_pt': {'x': 185, 'y': 238}},
                               {'id': 18, 'tick_pt': {'x': 210, 'y': 238}},
                               {'id': 19, 'tick_pt': {'x': 233, 'y': 238}},
                               {'id': 20, 'tick_pt': {'x': 258, 'y': 238}},
                               {'id': 21, 'tick_pt': {'x': 282, 'y': 238}},
                               {'id': 22, 'tick_pt': {'x': 304, 'y': 238}},
                               {'id': 23, 'tick_pt': {'x': 328, 'y': 238}},
                               {'id': 24, 'tick_pt': {'x': 352, 'y': 238}},
                               {'id': 25, 'tick_pt': {'x': 376, 'y': 238}},
                               {'id': 26, '